# Reto: competición de sistemas de clasificación

En este reto aplicaremos regresión logística a la tarea de clasificación [*california-housing*](https://scikit-learn.org/1.5/datasets/real_world.html#california-housing-dataset).

El objetivo de esta tarea es clasificar, con mínimo error, diferentes distritos del estado de California (EEUU) en 6 clases diferentes correspondientes a rangos de precios medios de vivienda, utilizando un conjunto de D=8 características numéricas.

## 1. Obtención del corpus y partición de datos

Ejecuta el siguiente bloque de código, en el que importamos las librerías necesarias, definimos constantes, obtenemos el dataset, barajamos (con una semilla concreta e inalterable) y particionamos los datos en train y test (con una proporción concreta e inalterable).

**MUY IMPORTANTE: NO MODIFICAR ESTE BLOQUE DE CÓDIGO, SOLO EJECUTAR.**

In [ ]:
### NO MODIFIQUES ESTE BLOQUE DE CÓDIGO; SOLO EJECÚTALO ###

import warnings; warnings.filterwarnings("ignore"); import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

RANDOM_SEED = 22
TEST_SIZE = 0.2

### NO MODIFIQUES ESTE BLOQUE DE CÓDIGO; SOLO EJECÚTALO ###

corp = fetch_california_housing()
X = corp.data.astype(np.float16) # muestras
y = corp.target.astype(np.uint)  # etiquetas de clase

### NO MODIFIQUES ESTE BLOQUE DE CÓDIGO; SOLO EJECÚTALO ###

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED)
D = X_train.shape[1]; C=np.unique(y_train).size; Ntr = X_train.shape[0]; Nte = X_test.shape[0];

print("******** INFORMACIÓN BÁSICA DEL DATASET **********")
print(f"D={D}, C={C}, N_train={Ntr}, N_test={Nte}")

### NO MODIFIQUES ESTE BLOQUE DE CÓDIGO; SOLO EJECÚTALO ###

******** INFORMACIÓN BÁSICA DEL DATASET **********
D=8, C=6, N_train=16512, N_test=4128


## 2. Entrenamiento y evaluación del sistema base (baseline)

A continuación podéis obtener una tasa de error de referencia, obtenido con un clasificador de regresión logística entrenado con `tol=0.01`, `C=1`, y `max_iter=10`. Este será nuestro sistema de clasificación base (baseline).

Vuestro objetivo será mejorar (lo máximo posible) la tasa de error obtenida con este sistema.

In [ ]:
# NOTA IMPORTANTE: SIEMPRE USAREMOS random_state=RANDOM_SEED
clf = LogisticRegression(random_state=RANDOM_SEED, C=1, tol=0.01, max_iter=10).fit(X_train, y_train)
err_test = (1 - accuracy_score(y_test, clf.predict(X_test)))*100
print(f'Error de clasificación en test (baseline): {err_test:5.1f}%')

Error de clasificación en test (baseline):  59.9%


## 3. Exploración/Optimización de hiperparámetros

Realiza una exploración y ajuste de los hiperparámetros tolerancia (`tol`), escalado del factor de regularización (`C`), y número de iteraciones máximas (`max_iter`), para minimizar la tasa de error de clasificación en test.

Reporta los resultados de los experimentos en una o varias tablas que muestren los valores de los tres hiperparámetros mencionados, además de las tasas de error de clasificación en train y test.

Introduce a continuación el código que hayas utilizado para realizar esta exploración/optimización, y asegúrate que el cuaderno conserva la salida de la ejecución de dicho código. Crea celdas de código adicionales si lo necesitas.

**IMPORTANTE: usa el parámetro `random_state=RANDOM_SEED` en `LogisticRegression()`**.

In [ ]:


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MaxAbsScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd

print(X_train.dtypes)

# 1. PREPROCESAMIENTO
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MaxAbsScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
#numerica
if isinstance(X_train, np.ndarray):
    preprocessor = ColumnTransformer(
        transformers=[('num', numeric_transformer, list(range(X_train.shape[1])))],
        remainder='passthrough'
    )
    #categorica
else:
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, make_column_selector(dtype_include=np.number)),
            ('cat', categorical_transformer, make_column_selector(dtype_include=['object', 'category']))
        ],
        remainder='passthrough'
    )

# 2. PIPELINE BASE
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=RANDOM_SEED))
])

# 3. GRID SEARCH
param_grid = [

    {
        'classifier__solver': ['lbfgs'],
        'classifier__C': [0.1, 1, 10, 100],
        'classifier__tol': [0.001, 0.0001],
        'classifier__max_iter': [2000],
        'classifier__class_weight': [None, 'balanced']
    },

    {
        'classifier__solver': ['liblinear'],
        'classifier__penalty': ['l2'],
        'classifier__C': [0.1, 1, 10, 100],
        'classifier__tol': [0.001],
        'classifier__max_iter': [2000],
        'classifier__class_weight': [None, 'balanced']
    }
]

# 4. EJECUCIÓN
print(f"Ejecutando (Seed={RANDOM_SEED})...")


grid = GridSearchCV(clf, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

# 5. RESULTADOS
best_params = grid.best_params_
best_model = grid.best_estimator_

err_train = (1 - best_model.score(X_train, y_train)) * 100
err_test = (1 - best_model.score(X_test, y_test)) * 100

print("\n" + "="*60)
print("RESULTADOS:")
print("="*60)
print(f"Factor C:           {best_params['classifier__C']}")
print(f"Tolerancia (tol):   {best_params['classifier__tol']}")
print(f"Iteraciones:        {best_params['classifier__max_iter']}")
print(f"Error TRAIN:        {err_train:.2f}%")
print(f"Error TEST:         {err_test:.2f}%")
print("="*60)


'''
numeric
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MaxAbsScaler())
])

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, list(range(X_train.shape[1])))],
    remainder='passthrough'
)'''

'''
categorica
numeric_transformer = Pipeline([...])
categorical_transformer = Pipeline([...])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, make_column_selector(dtype_include=np.number)),
        ('cat', categorical_transformer, make_column_selector(dtype_include=['object', 'category']))
    ],
    remainder='passthrough'
)



Ejecutando (Seed=22)...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

RESULTADOS:
Factor C:           100
Tolerancia (tol):   0.0001
Iteraciones:        2000
Error TRAIN:        42.10%
Error TEST:         43.36%


## 4. Determinación de los hiperparámetros óptimos y tasas de error

Por último, **modifica** la siguiente celda para que indique cuáles son los valores óptimos de los tres hiperparámetros `C`, `tol` y `max_iter`, así como las correspondientes tasas de error obtenidas en los conjuntos de entrenamiento y test.

- **Factor de regularización (`C`):**
- **Tolerancia (`tol`):**
- **Número máximo de iteraciones (`max_iter`):**
- **Tasa de error (%) obtenida en train:**
- **Tasa de error (%) obtenida en test:**


Nota: los valores de `C`, `tol` y `max_iter` que indiques aquí son los que usarás para participar en la evaluación final.